In [15]:
import bigframes.pandas as bpd

bpd.options.bigquery.ordering_mode = 'partial'


In [16]:
df = bpd.read_gbq("bigquery-public-data.baseball.schedules")[["homeTeamName", "awayTeamName", "duration_minutes"]]


In [17]:
import math

def moderate_cpu_work(n: int) -> int:
    """
    Performs a fixed amount of CPU-bound math to test UDF overhead.
    """
    if n is None:
        return 0
    
    # Using a fixed iteration count ensures the 'work' is consistent 
    # regardless of the input value, which helps in benchmarking.
    iterations = 100_000 
    aggregate = float(n)
    
    for i in range(iterations):
        # A mix of trig and power functions to keep the CPU busy
        # We use i to ensure the calculation doesn't get optimized away
        aggregate += math.sqrt(abs(math.sin(i) * math.cos(aggregate)))
        
    # Return a valid integer (modulo-ed to keep it clean)
    return int(aggregate) % 1_000_000

In [ ]:
classic_func = bpd.remote_function(reuse=False, cloud_function_service_account="default", cloud_function_memory_mib=16384)(moderate_cpu_work)
concurrent_func = bpd.remote_function(reuse=False, cloud_function_service_account="default", cloud_function_memory_mib=16384, workers=33, concurrency=32)(moderate_cpu_work)


In [ ]:
df1 = bpd.read_gbq("bigquery-public-data.bitcoin_blockchain.blockss")[["block_id"]]
r1 = df1.assign(result_val=df1["block_id"].apply(classic_func)).to_gbq()


/usr/local/google/home/tbergeron/src/bigframes/bigframes/core/logging/log_adapter.py:183: TimeTravelCacheWarning: Reading cached table from 2026-02-24 19:15:25.384225+00:00 to avoid
incompatibilies with previous reads of this table. To read the latest
version, set `use_cache=False` or close the current session with
Session.close() or bigframes.pandas.close_session().
  return method(*args, **kwargs)


In [ ]:
df2 = bpd.read_gbq("bigquery-public-data.bitcoin_blockchain.blocks")[["block_id"]]
r2 = df2.assign(result_val=df2["block_id"].apply(concurrent_func)).to_gbq()


/usr/local/google/home/tbergeron/src/bigframes/bigframes/core/logging/log_adapter.py:183: TimeTravelCacheWarning: Reading cached table from 2026-02-24 19:15:25.384225+00:00 to avoid
incompatibilies with previous reads of this table. To read the latest
version, set `use_cache=False` or close the current session with
Session.close() or bigframes.pandas.close_session().
  return method(*args, **kwargs)
